In [109]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError


In [110]:
# Load datasets
fake_politifact = pd.read_csv('politifact_fake.csv')
real_politifact = pd.read_csv('politifact_real.csv')
fake_gossipcop = pd.read_csv('gossipcop_fake.csv')
real_gossipcop = pd.read_csv('gossipcop_real.csv')

# Assign labels: 0 for fake news, 1 for real news
fake_politifact['label'] = 0
real_politifact['label'] = 1
fake_gossipcop['label'] = 0
real_gossipcop['label'] = 1

# Concatenate datasets
data = pd.concat([fake_politifact, real_politifact, fake_gossipcop, real_gossipcop], axis=0, ignore_index=True)


In [111]:
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert text to lowercase and tokenize words
    words = text.lower().split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)
    
    return cleaned_text

data['cleaned_title'] = data['title'].apply(clean_text)


In [112]:
# Split data into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [113]:
train_data.shape

(18556, 6)

In [114]:
train_data.head()


,id,news_url,title,tweet_ids,label,cleaned_title
9198,gossipcop-873539,https://www.etonline.com/news/223794_samuel_l_...,Samuel L. Jackson Schools James Corden in 'Dro...,897450356900114432\t897450895167729664\t897451...,1,samuel l jackson schools james corden drop mic...
14985,gossipcop-853226,https://www.nydailynews.com/new-york/nyc-crime...,Alleged Taylor Swift stalker waited on roof of...,864840870826090498\t864840973523623936\t864841...,1,alleged taylor swift stalker waited roof manha...
15100,gossipcop-850005,https://medium.com/@AlexChaney303138N/from-hop...,From Hopeful Australia Beginnings to a Tragic ...,864785411520049153\t864786362523820038\t864786...,1,hopeful australia beginnings tragic hollywood ...
6186,gossipcop-4516431341,variety.com/2017/legit/awards/tony-awards-2017...,Tony Awards 2017: Complete Winners List,873270018309447680\t873271704667205632\t873272...,0,tony awards complete winners list
11038,gossipcop-891466,https://en.wikipedia.org/wiki/Disappearance_of...,Disappearance of Madeleine McCann,926802996821970944\t926803044611829760\t926804...,1,disappearance madeleine mccann


In [115]:
test_data.shape

(4640, 6)

In [116]:
test_data.head()

,id,news_url,title,tweet_ids,label,cleaned_title
4291,gossipcop-3430607867,www.today.com/video/mariah-careys-twins-steal-...,Mariah Carey’s twins steal spotlight at Hollyw...,629276787600560128\t629286113706795009\t629326...,0,mariah carey twins steal spotlight hollywood w...
14922,gossipcop-930854,https://people.com/tv/teen-mom-bombshells/,Teen Mom's Most Bombshell and Dramatic Moments,991304765010333696\t991305082875662336\t991305...,1,teen mom bombshell dramatic moments
19868,gossipcop-911475,https://www.vanityfair.com/hollywood/2018/02/s...,S.N.L.: Watch Natalie Portman Rap About Star W...,960156003277856768\t960175040087150592\t960178...,1,n l watch natalie portman rap star wars preque...
12669,gossipcop-847921,https://www.springfieldspringfield.co.uk/view_...,The Arrangement (2017) s02e02 Episode Script,859786149417504770\t859786174889472000\t859787...,1,arrangement e episode script
12570,gossipcop-886689,https://en.paperblog.com/the-platinum-life-rec...,The Platinum Life Recap: The Ladies Take a Tri...,919764599016058880\t919764869817266176\t919765...,1,platinum life recap ladies take trip vegas aly...


In [117]:
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

glove_vectors = load_glove_vectors('glove.6B.50d.txt')


In [118]:
def weighted_average_vectors(docs, vectorizer, word_vectors, dim=50):
    # Calculate the tf-idf weights for the given documents
    tfidf_matrix = vectorizer.fit_transform(docs)

    # Initialize an empty matrix to store the weighted average vectors
    weighted_vectors = np.zeros((len(docs), dim))

    # Iterate through the documents and compute the weighted average vector for each
    for i, doc in enumerate(docs):
        words = doc.split()
        weighted_sum = np.zeros(dim)
        total_weight = 0

        for word in words:
            if word in word_vectors and word in vectorizer.vocabulary_:
                vector = word_vectors[word]
                weight = tfidf_matrix[i, vectorizer.vocabulary_[word]]
                weighted_sum += weight * vector
                total_weight += weight

        if total_weight != 0:
            weighted_vectors[i] = weighted_sum / total_weight

    return weighted_vectors


In [119]:
vectorizer = TfidfVectorizer(max_features=10000)
X_train = weighted_average_vectors(train_data['cleaned_title'], vectorizer, glove_vectors)
X_test = weighted_average_vectors(test_data['cleaned_title'], vectorizer, glove_vectors)

y_train = train_data['label'].values
y_test = test_data['label'].values


In [120]:
def create_autoencoder(input_dim, encoding_dim, hidden_layers):
    # Encoder
    encoder_input = Input(shape=(input_dim,))
    encoder_layers = [Dense(hidden_layers[0], activation='relu')(encoder_input)]

    for layer_size in hidden_layers[1:]:
        encoder_layers.append(Dense(layer_size, activation='relu')(encoder_layers[-1]))

    # Encoded representation
    encoded = Dense(encoding_dim, activation='relu')(encoder_layers[-1])

    # Decoder
    decoder_layers = [Dense(hidden_layers[-1], activation='relu')(encoded)]

    for layer_size in reversed(hidden_layers[:-1]):
        decoder_layers.append(Dense(layer_size, activation='relu')(decoder_layers[-1]))

    # Reconstruction
    decoder_output = Dense(input_dim, activation='linear')(decoder_layers[-1])

    # Build the autoencoder model
    autoencoder = Model(encoder_input, decoder_output)
    return autoencoder


In [121]:
input_dim = X_train.shape[1]
encoding_dim = 32
hidden_layers = [128, 64]

autoencoder = create_autoencoder(input_dim, encoding_dim, hidden_layers)
autoencoder.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 50)]              0         
                                                                 
 dense_18 (Dense)            (None, 128)               6528      
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 32)                2080      
                                                                 
 dense_21 (Dense)            (None, 64)                2112      
                                                                 
 dense_22 (Dense)            (None, 128)               8320      
                                                                 
 dense_23 (Dense)            (None, 50)                6450

In [122]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
autoencoder.compile(optimizer=optimizer, loss=loss)

# Train the model
real_headlines = X_train[y_train == 1]  # filter real news headlines
epochs = 100
batch_size = 32
autoencoder.fit(real_headlines, real_headlines, epochs=epochs, batch_size=batch_size, validation_split=0.1)


Epoch 1/100
393/393 [==============================] - 1s 1ms/step - loss: 0.0511 - val_loss: 0.0330
Epoch 2/100
393/393 [==============================] - 0s 1ms/step - loss: 0.0268 - val_loss: 0.0266
Epoch 3/100
393/393 [==============================] - 0s 1ms/step - loss: 0.0231 - val_loss: 0.0245
Epoch 4/100
393/393 [==============================] - 0s 955us/step - loss: 0.0220 - val_loss: 0.0234
Epoch 5/100
393/393 [==============================] - 0s 894us/step - loss: 0.0214 - val_loss: 0.0230
Epoch 6/100
393/393 [==============================] - 0s 944us/step - loss: 0.0210 - val_loss: 0.0227
Epoch 7/100
393/393 [==============================] - 0s 961us/step - loss: 0.0207 - val_loss: 0.0222
Epoch 8/100
393/393 [==============================] - 0s 1ms/step - loss: 0.0204 - val_loss: 0.0222
Epoch 9/100
393/393 [==============================] - 0s 1ms/step - loss: 0.0203 - val_loss: 0.0217
Epoch 10/100
393/393 [==============================] - 0s 940us/step - loss: 0.020

In [123]:
# Reconstruct headlines in the test set
X_test_reconstructed = autoencoder.predict(X_test)

# Calculate the reconstruction errors
reconstruction_errors = np.mean((X_test - X_test_reconstructed)**2, axis=1)


145/145 [==============================] - 0s 672us/step


In [124]:
from sklearn.metrics import confusion_matrix

def find_optimal_threshold(errors, y_true):
    min_diff = float('inf')
    optimal_threshold = 0

    for threshold in np.linspace(errors.min(), errors.max(), num=1000):
        y_pred = (errors > threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        fpr = fp / (fp + tn)  # False Positive Rate
        fnr = fn / (fn + tp)  # False Negative Rate

        diff = abs(fpr - fnr)

        if diff < min_diff:
            min_diff = diff
            optimal_threshold = threshold

    return optimal_threshold

threshold = find_optimal_threshold(reconstruction_errors, y_test)


In [125]:
y_pred = (reconstruction_errors > threshold).astype(int)


In [126]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Autoencoder Classifier Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")



Autoencoder Classifier Metrics:
Accuracy: 0.5174568965517241
Precision: 0.7593448131037379
Recall: 0.5204375359815774
F1 Score: 0.6175918018787361


In [127]:
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model with an increased number of iterations
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred_log_reg = log_reg.predict(X_test)

# Calculate metrics
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

print(f"Logistic Regression Metrics:\nAccuracy: {accuracy_log_reg}\nPrecision: {precision_log_reg}\nRecall: {recall_log_reg}\nF1 Score: {f1_log_reg}")


Logistic Regression Metrics:
Accuracy: 0.7676724137931035
Precision: 0.7801683816651076
Recall: 0.9602763385146805
F1 Score: 0.8609032258064516


In [128]:
from sklearn.ensemble import RandomForestClassifier

# Train the random forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest Metrics:\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nF1 Score: {f1_rf}")


Random Forest Metrics:
Accuracy: 0.7887931034482759
Precision: 0.7873271889400921
Recall: 0.9835924006908463
F1 Score: 0.8745840798566675
